In [1]:
import requests

url = "https://mediadive.dsmz.de/strains"
response = requests.get(url)

if response.status_code == 200:
    print("Successfully fetched the webpage")
else:
    print("Failed to fetch the webpage")

In [2]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(response.content, 'html.parser')
print(soup.prettify())  # This will print the HTML structure of the page

In [4]:
import requests
from bs4 import BeautifulSoup

# Example Name Link
example_name_link = "https://mediadive.dsmz.de/strains/view/DSM%201"  # Replace with an actual link

# Function to fetch and print the HTML structure of a given URL
def fetch_html_structure(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        print(soup.prettify())  # Print the formatted HTML structure
    except requests.exceptions.RequestException as e:
        print(f"Error fetching HTML content from {url}: {e}")

# Fetch and print the HTML structure of the example Name Link
fetch_html_structure(example_name_link)

In [32]:
import requests
from bs4 import BeautifulSoup

# Example Name Link
example_name_link = "https://mediadive.dsmz.de/strains/view/DSM%201"  # Replace with an actual link

# Function to fetch and print the HTML structure of a given URL
def fetch_html_structure(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        return soup
    except requests.exceptions.RequestException as e:
        print(f"Error fetching HTML content from {url}: {e}")
        return None

# Function to extract key data from the HTML structure
def extract_key_data(soup):
    # Extracting the title
    title = soup.find('title').text.strip() if soup.find('title') else 'N/A'

    # Extracting the strain name
    strain_name = soup.find('h2').text.strip() if soup.find('h2') else 'N/A'

    # Extracting the synonyms
    synonyms = []
    paragraphs = soup.find_all('p')
    for p in paragraphs:
        bold_text = p.find('b', string='Synonyms:')
        if bold_text:
            for content in p.contents:
                if content.name == 'a':
                    synonyms.append(content.get_text(strip=True))
                    synonyms.append('href: '+content.get('href'))
                elif isinstance(content, str) and content.strip():
                    synonyms.extend(content.split(', '))
            break
    
    # Extracting growth media details
    media_details = []
    media_boxes = soup.find_all('div', class_='box')

    for box in media_boxes:
        media_title = box.find('h3', class_='title').text.strip() if box.find('h3', class_='title') else 'N/A'
        media_link = box.find('a', class_='link colorless')['href'] if box.find('a', class_='link colorless') else 'N/A'
        
        # Corrected logic to check for growth observation
        growth_observed = 'Yes' if box.find('i', class_='ph ph-lg ph-check text-success') else 'No'
        
        growth_conditions = box.find('span', class_='badge danger').text.strip() if box.find('span', class_='badge danger') else 'N/A'
        
        media_details.append({
            'media_title': media_title,
            'media_link': media_link,
            'growth_observed': growth_observed,
            'growth_conditions': growth_conditions
        })

    # Displaying the extracted data
    print(f"Title: {title}")
    print(f"Strain Name: {strain_name}")
    print(f"Synonyms: {', '.join(synonyms)}")
    print("Media Details:")
    for media in media_details:
        print(f"  - Media Title: {media['media_title']}")
        print(f"    Media Link: {media['media_link']}")
        print(f"    Growth Observed: {media['growth_observed']}")
        print(f"    Growth Conditions: {media['growth_conditions']}")

# Fetch the HTML structure of the example Name Link
soup = fetch_html_structure(example_name_link)

# Extract and print key data if the HTML was successfully fetched
if soup:
    extract_key_data(soup)